In [1]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
import os

from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from huggingface_hub import notebook_login
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.text_splitter import CharacterTextSplitter
import textwrap
import sys
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

## RetrievalQA & Chat

In [9]:
#input_doc_pth = r'D:\nu_QA_data\m460bsp_Library_StdDriver_headers_1000'
#input_doc_pth = r'D:\nu_QA_data\m460bsp_StdDriver'
#input_doc_pth = r'D:\nu_QA_data\m2351bsp_1000'
#input_doc_pth = r'D:\nu_QA_data\m2351bsp_StdDriver_regs_1000'
#input_doc_pth = r'D:\nu_QA_data\m251bsp_StdDriver_1000'
input_doc_pth = r'D:\nu_QA_data\m251bsp_partial'
# load embedding model
print("===== Load the embedding model =====")
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',model_kwargs={'device': 'cpu'})

# Create vectors store
print("===== Build FAISS =====")
#vectorstore=FAISS.from_documents(texts, embeddings)
vectorstore=FAISS.load_local(input_doc_pth, embeddings)

===== Load the embedding model =====
===== Build FAISS =====


In [3]:
# From local location
#loc = r'C:\Users\USER\Desktop\llma\text-generation-webui\models\stabilityai_StableBeluga-7B'
#loc = r'C:\Users\USER\Desktop\llma\text-generation-webui\models\Llama-2-7b-chat-hf'
#loc = r'meta-llama/Llama-2-7b-hf'
#loc = r'C:\Users\USER\Desktop\llma\text-generation-webui\models\togethercomputer_LLaMA-2-7B-32K'
#loc = r'meta-llama/Llama-2-13b-chat-hf'
#loc = r'C:\Users\USER\Desktop\llma\text-generation-webui\models\Llama-2-7B-32K-Instruct'  #Bad
#loc = r'D:\stablecode-completion-alpha-3b-4k'
#loc = r'D:\codegen25-7b-multi'
#loc = r'D:\codegen25-7b-multi'
#loc = r'D:\CodeLlama-7b-hf'
#loc = r'D:\CodeLlama-7b-Instruct-hf'
loc = r'C:\Users\USER\Desktop\llma\text-generation-webui\models\CodeLlama-34b-Instruct-hf'

quantization_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_compute_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(loc)
model = AutoModelForCausalLM.from_pretrained(
    loc, device_map="auto",
    quantization_config=quantization_config,
)

#tokenizer = AutoTokenizer.from_pretrained(loc, trust_remote_code=True)
#model = AutoModelForCausalLM.from_pretrained(loc, device_map='auto',
#                       torch_dtype=torch.float16,
#                       use_auth_token=True,
#                       trust_remote_code=True,                      
#                       #load_in_8bit=True,
#                        #load_in_4bit=True
#                       )

#pipe = pipeline(
#         "text-generation",
#         #"question-answering",
#         model=model,
#         tokenizer= tokenizer,
#         torch_dtype=torch.bfloat16,
#         device_map="auto",
#         max_new_tokens = 1024,
#         do_sample=True,
#         top_k=10,
#         num_return_sequences=1,
#         eos_token_id=tokenizer.eos_token_id
#         )

# code llama
pipe = pipeline(
         "text-generation",
         #"question-answering",
         model=model,
         tokenizer= tokenizer,
         torch_dtype=torch.bfloat16,
         device_map="auto",
         max_new_tokens = 4096,
         #max_length=500,
         do_sample=True,
         top_k=10,
         top_p=0.9,
         temperature=0.1,
         num_return_sequences=1,
         eos_token_id=tokenizer.eos_token_id
         )

#pipe = pipeline(
#    text-generation",
#         #"question-answering",
#    model=model,
#    tokenizer= tokenizer,
#    do_sample=True,
#    top_k=10,
#    temperature=0.1,
#    top_p=0.95,
#    num_return_sequences=1,
#    eos_token_id=tokenizer.eos_token_id,
#    max_length=200,
#)

llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin C:\ProgramData\anaconda3\envs\llama\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll
CUDA SETUP: CUDA runtime path found: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary C:\ProgramData\anaconda3\envs\llama\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll...


C:\ProgramData\anaconda3\envs\llama\Lib\site-packages\bitsandbytes\cuda_setup\main.py:156: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/ProgramData/anaconda3/envs/llama/bin')}
  warn(msg)
C:\ProgramData\anaconda3\envs\llama\Lib\site-packages\bitsandbytes\cuda_setup\main.py:156: UserWarning: C:\ProgramData\anaconda3\envs\llama did not contain ['cudart64_110.dll', 'cudart64_120.dll', 'cudart64_12.dll'] as expected! Searching further paths...
  warn(msg)


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate

# Prompt
#template="""Use the following pieces of context to answer the question at the end.
#Please answer with C Code function as complete as possible.
#If you don't know the answer or the question has nothing to do with code or programing, don't try to make up an answer.
#{context}
#Question: {question}
#Answer:"""

# Prompt
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
system_template = """You will use the provided context to answer the user questions.
Read the given context before answering with C Code function as complete as possible.
"""
instruction ="""
context: {context}
User: {question}"""

def prompt_format(instruction= instruction, system_template= system_template):
    SYSTEM_PROMPT = B_SYS + system_template + E_SYS
    prompt_templte = B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_templte
template=prompt_format()

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)

## RetrievalQA

In [10]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", return_source_documents=True, retriever=vectorstore.as_retriever(), chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    })

#chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", return_source_documents=True, retriever=vectorstore.as_retriever(
#    search_type="mmr", # Also test "similarity"
#    search_kwargs={"k": 8}), 
#    chain_type_kwargs={
#        "prompt": PromptTemplate(
#            template=prompt_format(),
#            input_variables=["context", "question"],
#        ),
#    })

In [14]:
#query = "Write a C code function that reverse a string, input a string in parameter and return the reversed string"

#query = "Write a C code that BMC data transfer with PDMA"
#query = "Write a C code of ACMP comparing DAC output with ACMP1_P1"
query = "Generate a C source code to get data from UART, PDMA to memory address 0x20000000, and do CRC32 at 0x20000000"
#query = "Write an example C code of configure BMC"

#query = "Generate a C source code which open EBI with bank2, 16BIT width, and fast timing"
#query = "Generate a C source code which TIMER0 MODE is TIMER_ONESHOT_MODE, TIMER0_FREQ is 1000000, TIMER0_PRESCALE_VALUE is 5, TIMER0_CMP_VALUE is 0x5A5A5A"
result=chain({"query": query}, return_only_outputs=True)
#wrapped_text = textwrap.fill(result['result'], width=500)
#print(wrapped_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [15]:
for i in range(len(result['source_documents'])):
    print(result['source_documents'][i].metadata)

{'source': 'D:\\nu_QA_data\\m251bsp_partial_source\\CRC_CRC32\\main.c', 'language': <Language.C: 'c'>}
{'source': 'D:\\nu_QA_data\\m251bsp_partial_source\\CRC_CRC32\\main.c', 'language': <Language.C: 'c'>}
{'source': 'D:\\nu_QA_data\\m251bsp_partial_source\\UART_PDMA\\main.c', 'language': <Language.C: 'c'>}
{'source': 'D:\\nu_QA_data\\m251bsp_partial_source\\crc.h', 'language': <Language.C: 'c'>}


In [16]:
print(result['result'])

  Here is a C source code that performs the described operations:
```c
#include <stdio.h>
#include <string.h>
#include "sys.h"
#include "uart.h"
#include "pdma.h"
#include "crc.h"

#define PDMA_TEST_LENGTH  1024
#define PDMA_TEST_ADDRESS 0x20000000

volatile uint32_t g_au8RxBuffer[PDMA_TEST_LENGTH];

void UART1_IRQHandler(void)
{
    uint8_t u8InChar;
    uint32_t u32Address;
    uint32_t u32Size;
    uint32_t u32FMCChecksum, u32CRC32Checksum, u32PDMAChecksum;

    /* Get the address and size of the data to be transferred */
    u32Address = PDMA_TEST_ADDRESS;
    u32Size = PDMA_TEST_LENGTH;

    /* Disable CRC function */
    CRC->CTL = 0;

    /* Enable PDMA module clock */
    CLK->AHBCLK |= CLK_AHBCLK_PDMACKEN_Msk;

    /* Give valid source address and transfer count and program PDMA memory to memory, dest => CRC_WDATA */
    PDMA->CHCTL = (1 << 0); // use PDMA CH0
    DMA_DESC[0].ctl =
        (1 << PDMA_DSCT_CTL_OPMODE_Pos)  | (0 << PDMA_DSCT_CTL_TXTYPE_Pos) |
        (7 << PDMA_

## Another choose load_qa_chain()

In [5]:

# Docs Retriever
#retriever=vectorstore.as_retriever(search_type="mmr", # Also test "similarity"
#                                   search_kwargs={"k": 8})
retriever=vectorstore.as_retriever(search_type="similarity", # Also test "similarity"
                                   search_kwargs={"k": 8})

# question & vector search
question = "Write a C code of ACMP comparing DAC output with ACMP1_P1"
docs = retriever.get_relevant_documents(question)

# Chain
chain = load_qa_chain(llm, chain_type="stuff", prompt=QA_CHAIN_PROMPT) 

In [ ]:
result = chain({"input_documents": docs, "question": question}, return_only_outputs=True)

In [ ]:
print(len(docs))
print(result['output_text'])

## ConversationSummaryMemory

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectorstore.as_retriever(), memory=memory)

In [ ]:
#query = "Write a C code of ACMP comparing DAC output with ACMP1_P1"
#query = "Write a C code of BMC data transfer with PDMA"
#query = "Write a C code function that reverse a string, input a string in parameter and return the reversed string"
query = "Give me a C code of configure BMC example"
result = qa(query)

In [ ]:
print(result['answer'])